<a href="https://colab.research.google.com/github/ejdogar/NLP/blob/main/With_Sentences_RAG_Assignment_4_Ejaz_ul_Haq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

In [73]:
dataset = pd.read_csv("/content/drive/MyDrive/Atomcamp/Assignments/Assignment-4/Pak Suzuki Car Reviews.csv")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


# Step 1: Data Preprocessing

#### 1.1 Converting Date Column to DateTime Format

In [74]:
dataset["Year"] = pd.to_datetime(dataset["Date"]).dt.year
dataset.drop("Date", axis=1, inplace = True)
dataset.head()

,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,0,2023 Suzuki Bolan,1,\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1,2023
1,1,2022 Suzuki Swift GLX CVT,4,"\nA good looking car with best fuel ? economy,...",5,4,5,4,5,2023
2,2,2022 Suzuki Cultus Auto Gear Shift,5,\nfor me i would must buy because its will sav...,5,5,5,5,5,2023
3,3,2007 Suzuki Liana RXi (CNG),5,\nI have used it for more than 5 years. litera...,5,5,5,5,5,2023
4,4,2022 Suzuki Alto VX,4,\nExterior: 10/10 on design but the quality us...,5,5,5,4,5,2023


#### 1.2 Creating a Year Column
Splitting the Model Column into Model Name and Year Column, in order to enhance model answering capability.

In [45]:
# dataset["Year"], dataset["Model"] = dataset["Model"].str.split(" ").str[0], dataset["Model"].str.split(" ").str[1:].str.join(" ")
# dataset.head()

#### 1.3 Formatting our reviews

In [75]:
dataset["Review"] = dataset["Review"].str.replace("\n", "")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,0,2023 Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023
1,1,2022 Suzuki Swift GLX CVT,4,"A good looking car with best fuel ? economy,Dr...",5,4,5,4,5,2023
2,2,2022 Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023
3,3,2007 Suzuki Liana RXi (CNG),5,I have used it for more than 5 years. literall...,5,5,5,5,5,2023
4,4,2022 Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023


In [76]:
import re
for index, review in enumerate(dataset['Review']):
  review = re.sub("[?.,$]", " ", review)
  dataset["Review"][index] = review
dataset.head()


<ipython-input-76-2636c59a88a7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Review"][index] = review


,Unnamed: 0,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,0,2023 Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023
1,1,2022 Suzuki Swift GLX CVT,4,A good looking car with best fuel economy Dr...,5,4,5,4,5,2023
2,2,2022 Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023
3,3,2007 Suzuki Liana RXi (CNG),5,I have used it for more than 5 years literall...,5,5,5,5,5,2023
4,4,2022 Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023


#### 1.4 Dropping Unnamed Column
As this column just have indexes and is of no significance as we have system indexing available for out dataset.

In [77]:
dataset.drop("Unnamed: 0", axis=1, inplace=True)
dataset.head()

,Model,Overall_rating,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,2023 Suzuki Bolan,1,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1,2023
1,2022 Suzuki Swift GLX CVT,4,A good looking car with best fuel economy Dr...,5,4,5,4,5,2023
2,2022 Suzuki Cultus Auto Gear Shift,5,for me i would must buy because its will save ...,5,5,5,5,5,2023
3,2007 Suzuki Liana RXi (CNG),5,I have used it for more than 5 years literall...,5,5,5,5,5,2023
4,2022 Suzuki Alto VX,4,Exterior: 10/10 on design but the quality used...,5,5,5,4,5,2023


1.5 Coverting Rating from Numbers to Wrod Representation

In [49]:
# rating_dic = {1: "Worst", 2: "Bad", 3: "Average", 4: "Good", 5: "Excellent"}
# dataset[["Overall_rating", "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]] = dataset[["Overall_rating",  "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]].applymap(rating_dic.get)
# dataset.head()

#### 1.6 Converting Dataframe into Sentences
Here I am converting all the data available in the dataframe into sentences, where each row would be represented by a unique sentence. Doing this will help our model learn better semantics.

In [78]:
sentences = []
for index, row in dataset.iterrows():
  sentences.append(f"In year {row[8]} someone reviewed {row[0]} with an over all rating of {row[1]} and rated Style, Comfort, Fuel Economy, Performance, Value for Money as {row[3]}, {row[4]}, {row[5]}, {row[6]}, {row[7]} respectively. Review from this customer was {row[2]}")
print(len(sentences))
print(sentences[0])

1099
In year 2023 someone reviewed 2023 Suzuki Bolan with an over all rating of 1 and rated Style, Comfort, Fuel Economy, Performance, Value for Money as 1, 1, 1, 1, 1 respectively. Review from this customer was is gadi k baray me 3 lafaz kahunga 1 bakwaaas 2 bakwaaaaaaas 3 bakwaaaassaaaaaaaaasfazool tareen or gharya tareen gadi shame on Suzuki company dunya me har cheez badal gai comfort level kahan se kahan ponch gya par ye farsoooda gadi he koi isko lyne ka sochy bhi na


In [79]:
models = []
for index, row in dataset.iterrows():
  models.append(f"{row[0]}")
print(len(models))
print(models[0])

1099
2023 Suzuki Bolan


In [81]:
dataframe = pd.DataFrame(models, columns=["Model"])
dataframe["Review"] = sentences
dataframe.head()

,Model,Review
0,2023 Suzuki Bolan,In year 2023 someone reviewed 2023 Suzuki Bola...
1,2022 Suzuki Swift GLX CVT,In year 2023 someone reviewed 2022 Suzuki Swif...
2,2022 Suzuki Cultus Auto Gear Shift,In year 2023 someone reviewed 2022 Suzuki Cult...
3,2007 Suzuki Liana RXi (CNG),In year 2023 someone reviewed 2007 Suzuki Lian...
4,2022 Suzuki Alto VX,In year 2023 someone reviewed 2022 Suzuki Alto...


### Importing Necessary Libraries For Our Model

In [83]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [88]:
!pip install accelerate==0.22.0 peft==0.5.0 bitsandbytes==0.40.2 transformers==4.31.0 langchain chromadb --quiet
!pip install tiktoken --quiet
!pip install sentence_transformers lark auto-gptq optimum --quiet

# Step 2: Creating Data Chunk From our CSV File

In [89]:
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(dataframe, page_content_column="Model")
documents = loader.load()
documents[0].page_content

'2023 Suzuki Bolan'

In [90]:
documents[0].metadata

{'Review': 'In year 2023 someone reviewed 2023 Suzuki Bolan with an over all rating of 1 and rated Style, Comfort, Fuel Economy, Performance, Value for Money as 1, 1, 1, 1, 1 respectively. Review from this customer was is gadi k baray me 3 lafaz kahunga 1 bakwaaas 2 bakwaaaaaaas 3 bakwaaaassaaaaaaaaasfazool tareen or gharya tareen gadi shame on Suzuki company dunya me har cheez badal gai comfort level kahan se kahan ponch gya par ye farsoooda gadi he koi isko lyne ka sochy bhi na'}

# Step 3: Creating Embeddings of Our Chunks
Generating Embaddings based on our above created chunks.

In [92]:
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "all-MiniLM-L6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

In [62]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00


# Step 4: Storing Embeddings in the Vector DB.

In [93]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents, embeddings)

In [94]:
import torch
import transformers
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, GPTQConfig

In [95]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/llama-2-7b-Chat-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                        )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [96]:
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 256,
                do_sample=True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.2
                )

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [97]:
print(llm(prompt="What are the reviews regarding Suzuki Alto in year 2015?"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



A: The Suzuki Alto is a popular subcompact car that has been around since 1974. In 2015, the Alto received positive reviews for its fuel efficiency, affordability, and reliability. Here are some of the reviews from reputable sources:
* Car and Driver gave the 2015 Suzuki Alto a rating of 6 out of 10, praising its good fuel economy but criticizing its lackluster performance and limited features.
* Kelley Blue Book awarded the Alto with an overall score of 4.3 out of 5, citing its excellent fuel economy, spacious interior, and low price point as major advantages. However, they also noted that the Alto's engine can be noisy and underpowered at times.
* Edmunds gave the 2015 Suzuki Alto a rating of B+, highlighting its impressive fuel economy and affordability while noting that it may not offer enough features to appeal to more discerning buyers.
* Consumer Reports rated the Alto 4 out of 5, praising its fuel efficiency and value but


In [98]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
        llm=llm, chain_type='stuff',
        retriever=db.as_retriever()
        )

In [99]:
print(rag("What are the reviews regarding Suzuki Alto in year 2015?")["result"])

 The Suzuki Alto is a popular subcompact car produced by Suzuki Motor Corporation since 1974. In 2015, there were several models available with different features and specifications. Here are some reviews for each model:
Suzuki Alto L (2015): This is the base model of the series, which comes with a 660cc engine and offers good fuel efficiency. However, it lacks advanced safety features like airbags and ABS. Reviewers praised its affordability but noted that it doesn't have enough power for highway driving. Rating: 3/5 stars
Suzuki Alto Lapin 10th Anniversary Limited (2015): This is a special edition model released to celebrate the 10th anniversary of the Alto series. It has a more powerful 1.0-liter engine and additional features like side airbags and Bluetooth connectivity. Reviewers appreciated its premium feel and better performance but felt that it was overpriced compared to other models. Rating: 4/5 stars
Suzuki Alto F (2015): This is another variant
